In [33]:
#import pixiedust
import numpy

In [34]:
%cd .
%run visualisation.ipynb
%run read_graph.ipynb

g = read_nba_full("../Data/Basketball/Players.csv",
                  "../Data/Basketball/Edges.csv",
                  False, True)

all11_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588, 2586,4198,115,2925,1459,2235,1794,415,1460,564,4168,2384,1142,3134,1849,2145,1945,3283,3669,1446,718,1031,1898,2577,2492,3502,620,3079,3672,1995,3383,4609,4090,65,2930,1504]
nba11_20 = g.subgraph(all11_20)

all15_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588, 2586,4198,115,2925,1459,2235,1794,415,1460,564,4168,2384,1142]
nba15_20 = g.subgraph(all15_20)

all19_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159]
nba19_20 = g.subgraph(all19_20)

filmweb = read_filmweb_full("../Data/Filmweb/allActors.csv",
                  "../Data/Filmweb/allActorsRelation.csv",
                  False, False, False, False)

/home/bartek/Pulpit/community-detection/Implementation
Reading full NBA graph, begin...
Reading file ../Data/Basketball/Players.csv, begin...
Data lines read: 4800
Reading file ../Data/Basketball/Edges.csv, begin...
Data lines read: 146369
Reading NBA vertices, begin...
Vertices read: 4800
Reading NBA edges, begin...
Weights loaded correctly
Edges read: 146369
Loading succesfull
Reading full Filmweb graph, begin...
Reading file ../Data/Filmweb/allActors.csv, begin...
Data lines read: 10000
Reading file ../Data/Filmweb/allActorsRelation.csv, begin...
Data lines read: 190031
Reading Filmweb vertices, begin...
Vertices read: 10000
Reading Filmweb edges, begin...
Edges read: 190031
Loading succesfull


In [42]:
def modularity(graph, community):
    M_in = 0
    M_out = 0
    for node in community:
        edges = graph.incident(node)
        for e in edges:
            edge = graph.es[e]
            if node == edge.target_vertex and node == edge.source_vertex:
                print("Warn - out vertex equals current vertex")
            elif node == edge.target_vertex:
                if edge.source_vertex in community:
                    M_in+=1
                else:
                    M_out+=1
            elif node == edge.source_vertex:
                if edge.target_vertex in community:
                    M_in+=1
                else:
                    M_out+=1
    if M_out == 0:
        return -1
    return M_in / 2 / M_out


In [43]:
def step1(graph):
    print("basic ocdlce begin ...")
    communities = []
    for v in graph.vs:
        v["communities"] = []
    for u in graph.vs:
        for v in graph.vs:
            if len(set(u["communities"]).intersection(v["communities"])) == 0 and u!=v:
                tmp_comm = [u,v]
                NC = list(set(graph.vs[graph.neighbors(v)]).union(graph.vs[graph.neighbors(u)]))
                if len(NC) > 2:
                    for node in NC:
                        if modularity(graph, tmp_comm + [node]) > modularity(graph, tmp_comm):
                            tmp_comm = tmp_comm + [node]
                if len(tmp_comm) > 4:
                    id = len(communities)+1
                    communities.append({"nodes": tmp_comm, "id": id })
                    for node in tmp_comm:
                        node["communities"].append(id)
    return graph, communities


In [44]:
def draw_basic_ocdlce(graph,output):
    g, com = step1(graph)
    plot_full(g, output, 20)

In [45]:
draw_basic_ocdlce(nba11_20, "ocdlce.svg")

In [46]:
def E_in(nodes):
    edges = []
    for node in nodes:
        for edge in node.all_edges():
            if node == edge.target_vertex and node == edge.source_vertex:
                print("Warn - out vertex equals current vertex")
            elif node == edge.target_vertex:
                if edge.source_vertex in nodes:
                    edges.append(edge)
            elif node == edge.source_vertex:
                if edge.target_vertex in nodes:
                    edges.append(edge)
    return edges

In [47]:
def WOS(graph, com1, com2, alfa, beta):
    v = len(set(com1["nodes"]).intersection(com2["nodes"]))
    min_v = min(len(com1["nodes"]),len(com2["nodes"]))
    e1 = E_in(com1["nodes"])
    e2 = E_in(com2["nodes"])
    e = len(set(e1).intersection(e2))
    min_e = min(len(e1),len(e2))
    res = alfa * v / min_v + (1-alfa)*e/min_e >= beta
    return res

In [48]:
def Union(graph, all_coms, com, alfa, beta):
    res = []
    for c in all_coms:
        if c["id"] != com["id"]:
            if WOS(graph, c, com, alfa, beta):
                res.append(c)
    return res

In [49]:
def step2(graph, local_communities, alfa, beta):
    C = local_communities
    for com in C:
        res = Union(graph, C, com, alfa, beta)
        for c in res:
            for node in c["nodes"]:
                node["communities"].remove(c["id"])
                if com["id"] not in node["communities"]:
                    node["communities"].append(com["id"])
            C.remove(c)
    return graph, C

In [50]:
def draw_merged_ocdlce(graph,output, alfa, beta):
    g, com = step1(graph)
    print("Communities after step1: " + str(len(com)))
    g, com = step2(g, com, alfa, beta)
    print("Communities after step2: " + str(len(com)))
    plot_full(g, output, 20)

In [51]:
def step3(graph):
    for v in graph.vs:
        if len(v["communities"]) == 0:
            print("ok")
            coms = {}
            for u in graph.neighbors(v):
                coms = coms.union(graph.vs[graph.neighbors(u)]["communities"])
            for com in coms:
                if modularity(graph, com + [v]) > modularity(graph, com):
                    v["communities"].append(com["id"])
    return graph


In [52]:
draw_merged_ocdlce(nba11_20, "ocdlce2.svg", 0.5, 0.5)

basic ocdlce begin ...
Communities after step1: 239
Communities after step2: 64
Plotting graph with 83 vertices and 315 edges, begin...
Plotting sucessful


In [ ]:
def draw_full_ocdlce(graph,output, alfa, beta):
    g, com = step1(graph)
    print("Communities after step1: " + str(len(com)))
    g, com = step2(g, com, alfa, beta)
    print("Communities after step2: " + str(len(com)))
    g = step3(g)
    plot_full(g, output, 20)

In [ ]:
draw_full_ocdlce(filmweb, "filmweb.svg", 0.5, 0.5)